# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

In [2]:
from fireplace import cards, exceptions, utils
from fireplace.player import Player
from fireplace.game import Game
from fireplace.deck import Deck
from fireplace.utils import get_script_definition, random_draft

In [3]:
from hearthstone.enums import PlayState, Step, Mulligan, State, CardClass, Race, CardSet, CardType

In [4]:
import re
import string
import random

In [5]:
cards.db.initialize()

[fireplace.__init__]: Initializing card database
[fireplace.__init__]: Merged 25793 cards


In [6]:
GREEN = "\033[92m"
RED = "\033[91m"
ENDC = "\033[0m"
PREFIXES = {
    GREEN: "Implemented",
    RED: "Not implemented",
}
implemented_cards = []
unimplemented_cards = []

In [7]:
SOLVED_KEYWORDS = [
    "Windfury", "Charge", "Divine Shield", "Taunt", "Stealth", "Poisonous",
    r"Can't be targeted by spells or Hero Powers\.",
    r"Can't attack\.",
    "Destroy any minion damaged by this minion.",
    r"Your Hero Power deals \d+ extra damage.",
    r"Spell Damage \+\d+",
    r"Overload: \(\d+\)",
]

In [8]:
DUMMY_CARDS = (
    "PlaceholderCard",  # Placeholder Card
    "CS2_022e",  # Polymorph
    "EX1_246e",  # Hexxed
    "EX1_345t",  # Shadow of Nothing
    "GAME_006",  # NOOOOOOOOOOOO
    "LOEA04_27",  # Animated Statue
    "Mekka4e",  # Transformed
    "NEW1_025e",  # Bolstered (Unused)
    "TU4c_005",  # Hidden Gnome
    "TU4c_007",  # Mukla's Big Brother

    # Dynamic buffs set by their parent
    "CS2_236e",  # Divine Spirit
    "EX1_304e",  # Consume (Void Terror)
    "LOE_030e",  # Hollow (Unused)
    "NEW1_018e",  # Treasure Crazed (Bloodsail Raider)
)

Por algun motivo separo unas cartas que llamo IMPLEMENTADAS de unas cartas que llamo NO_IMPLEMENTADAS

In [9]:
for id in sorted(cards.db):
    card = cards.db[id]
    ret = card.description
    ret = re.sub("<i>.+</i>", "", ret)
    ret = re.sub("(<b>|</b>)", "", ret)
    ret = re.sub("(" + "|".join(SOLVED_KEYWORDS) + ")", "", ret)
    ret = re.sub("<[^>]*>", "", ret)
    exclude_chars = string.punctuation + string.whitespace
    ret = "".join([ch for ch in ret if ch not in exclude_chars])
    description = ret
    implemented = False

    if not description:
        # Minions without card text or with basic abilities are implemented
        implemented = True
    elif card.card_set == CardSet.CREDITS:
        implemented = True

    if id in DUMMY_CARDS:
        implemented = True

    carddef = get_script_definition(id)
    if carddef:
        implemented = True

    color = GREEN if implemented else RED
    name = color + "%s: %s" % (PREFIXES[color], card.name) + ENDC

    if implemented:
        implemented_cards.append(card.id)
    else:
        unimplemented_cards.append(card.id)

IMPLEMENTED_CARDS = len(implemented_cards)
UNIMPLEMENTED_CARDS = len(unimplemented_cards)

print("IMPLEMENTED CARDS: "+str(IMPLEMENTED_CARDS))
print("UNIMPLEMENTED CARDS: "+str(UNIMPLEMENTED_CARDS))

IMPLEMENTED CARDS: 7923
UNIMPLEMENTED CARDS: 17870


In [10]:
heroes=list(CardClass)
heroes.remove(CardClass.INVALID)
heroes.remove(CardClass.DREAM)
heroes.remove(CardClass.NEUTRAL)
#heroes.remove(CardClass.WHIZBANG)
heroes

hero1=random.choice(heroes)
print(hero1)
print(hero1.default_hero)

CardClass.MAGE
HERO_08


In [11]:
[hero1, CardClass.NEUTRAL]

[<CardClass.MAGE: 4>, <CardClass.NEUTRAL: 12>]

In [12]:
def get_collection(heroe):
    collection = []
    for card in cards.db.keys():
        if str(card) not in implemented_cards:
                continue
        cls = cards.db[card]
        if not cls.collectible:
            continue
        if cls.type == CardType.HERO:
            # Heroes are collectible...
            continue
        if cls.card_class and cls.card_class in [heroe, CardClass.NEUTRAL]:
            # Play with more possibilities
            collection.append(cls)
    count=0
    for j in collection:
        if j.card_class == heroe:
            count=count+1
    print("Totales Neutras + hero: ",len(collection))
    print("Cartas de heroe: ",count)
    return collection

In [13]:
def get_deck(collection):
    deck=[]
    while len(deck) < Deck.MAX_CARDS:
        card = random.choice(collection)
        if deck.count(card.id) < card.max_count_in_deck:
            deck.append(card.id)
    return deck

In [14]:
collection1=get_collection(hero1)

Totales Neutras + hero:  867
Cartas de heroe:  128


In [15]:
deck1=get_deck(collection1)
print(deck1)

['UNG_937', 'GIL_648', 'LOOT_521', 'LOOT_134', 'CFM_620', 'EX1_412', 'CFM_623', 'UNG_073', 'NEW1_023', 'CFM_655', 'EX1_020', 'OG_280', 'ICC_836', 'GIL_809', 'CS1_069', 'CS2_119', 'AT_115', 'LOE_086', 'LOOT_137', 'OG_042', 'LOOT_125', 'EX1_100', 'UNG_900', 'CFM_715', 'LOOT_152', 'GVG_122', 'GIL_681', 'LOOT_394', 'EX1_105', 'BOT_103']


In [16]:
hero2=random.choice(heroes)
print(hero2)
collection2=get_collection(hero2)
deck2=get_deck(collection2)
print(deck2)

CardClass.WHIZBANG
Totales Neutras + hero:  739
Cartas de heroe:  0
['CORE_ICC_913', 'UNG_099', 'LOOT_130', 'EX1_080', 'LOOT_375', 'UNG_079', 'BRM_031', 'PRO_001', 'OG_272', 'AT_118', 'NEW1_016', 'ICC_067', 'CFM_067', 'CS1_042', 'VAN_NEW1_023', 'VAN_CS2_142', 'CFM_854', 'GIL_121', 'OG_173', 'LOOT_150', 'GVG_114', 'UNG_818', 'BOT_021', 'NEW1_018', 'AT_103', 'GVG_106', 'AT_122', 'EX1_020', 'CS1_069', 'CFM_646']


In [17]:
from fireplace.player import Player
player1=Player("Player1", deck1, hero1.default_hero)
player2=Player("Player2", deck2, hero2.default_hero)

Vemos que inicialmente la mano y el deck de cada jugador están vacías:

In [18]:
player1.deck,player1.hand

(<Deck (0 cards)>, [])

In [19]:
player2.deck,player2.hand


(<Deck (0 cards)>, [])

In [20]:
game=Game(players=(player1, player2))
game.start()

[fireplace.entity]: Setting up game Game(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Fireblast')>)> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Fireblast')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Jaina Proudmoore')>)> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.actions]: Player1 summons [<Hero ('Jaina Proudmoore')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) shuffles their deck
[f

Después de iniciar el juego, ya tienen valores:

In [21]:
player1.deck,player1.hand

(<Deck (26 cards)>,
 [<Minion ('Fen Creeper')>,
  <Minion ("Y'Shaarj, Rage Unbound")>,
  <Minion ('Mountain Giant')>,
  <Minion ('Oasis Snapjaw')>])

In [22]:
player2.deck,player2.hand

(<Deck (27 cards)>,
 [<Minion ('Arcane Tyrant')>,
  <Minion ('Fen Creeper')>,
  <Minion ('Twilight Summoner')>])

Hasta Aquí se iniciliza la partida

In [30]:
print(player1.hand[0])
print(player1.hand[1])
print(player1.hand[2])
print(player1.hand[3])

print(player1.hand[0] in implemented_cards)
print(player1.hand[1] in implemented_cards)
print(player1.hand[2] in implemented_cards)
print(player1.hand[3] in implemented_cards)

print(implemented_cards.index(player1.hand[0]))
print(implemented_cards.index(player1.hand[1]))
print(implemented_cards.index(player1.hand[2]))
print(implemented_cards.index(player1.hand[3]))

Fen Creeper
Y'Shaarj, Rage Unbound
Mountain Giant
Oasis Snapjaw
True
True
True
True
1834
5464
2441
1940


Vamos a probar a ver si yo le doy la carta al jugador, si todo funciona

In [32]:
player1.give("EX1_414")
player1.hand

[fireplace.actions]: Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>) triggering <TargetedAction: Give(<Give.CARD>='EX1_414')> targeting [Player(name='Player1', hero=<Hero ('Jaina Proudmoore')>)]
[fireplace.actions]: Giving [<Minion ('Grommash Hellscream')>] to Player1
[fireplace.card]: <Minion ('Grommash Hellscream')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


[<Minion ('Fen Creeper')>,
 <Minion ("Y'Shaarj, Rage Unbound")>,
 <Minion ('Mountain Giant')>,
 <Minion ('Oasis Snapjaw')>,
 <Minion ('Arcane Tyrant')>,
 <Minion ('Grommash Hellscream')>]

In [33]:
print("EX1_414" in implemented_cards)
print(implemented_cards.index("EX1_414"))

True
2609


In [34]:
print(player1.hand[5])

print(player1.hand[5] in implemented_cards)

print(implemented_cards.index(player1.hand[5]))

Grommash Hellscream
True
2609


**Aquí añado Gromash a la mano compruebo el índice como lo hace el código y sigue funcionando**

In [35]:
player2.give('AT_044')
player2.hand

[fireplace.actions]: Player(name='Player2', hero=<Hero ('Whizbang the Wonderful')>) triggering <TargetedAction: Give(<Give.CARD>='AT_044')> targeting [Player(name='Player2', hero=<Hero ('Whizbang the Wonderful')>)]
[fireplace.actions]: Giving [<Spell ('Mulch')>] to Player2
[fireplace.card]: <Spell ('Mulch')> moves from <Zone.SETASIDE: 6> to <Zone.HAND: 3>
[fireplace.entity]: Empty stack, refreshing auras and processing deaths


[<Minion ('Arcane Tyrant')>,
 <Minion ('Fen Creeper')>,
 <Minion ('Twilight Summoner')>,
 <Spell ('Mulch')>]

Vamos a ver lo que hacen en los test de fireplace

In [41]:
import random

from hearthstone.enums import *

import fireplace.cards
from fireplace.brawls import *
from fireplace.game import BaseGame, CoinRules, Game
from fireplace.logging import log
from fireplace.player import Player
from fireplace.utils import random_draft


# Token minions
ANIMATED_STATUE = "LOEA04_27"
GOLDSHIRE_FOOTMAN = "CS1_042"
TARGET_DUMMY = "GVG_093"
KOBOLD_GEOMANCER = "CS2_142"
SPELLBENDERT = "tt_010a"
CHICKEN = "GVG_092t"
IMP = "EX1_598"
MURLOC = "PRO_001at"
WISP = "CS2_231"
WHELP = "ds1_whelptoken"

# Token spells
INNERVATE = "EX1_169"
MOONFIRE = "CS2_008"
PYROBLAST = "EX1_279"
CIRCLE_OF_HEALING = "EX1_621"
DREAM = "DREAM_04"
SILENCE = "EX1_332"
THE_COIN = "GAME_005"
HAND_OF_PROTECTION = "EX1_371"
MIND_CONTROL = "CS1_113"
TIME_REWINDER = "PART_002"
SOULFIRE = "EX1_308"
UNSTABLE_PORTAL = "GVG_003"
HOLY_LIGHT = "CS2_089"

# Token weapon
LIGHTS_JUSTICE = "CS2_091"

# Fandral Staghelm
FANDRAL_STAGHELM = "OG_044"

# Collectible cards excluded from random drafts
BLACKLIST = (
	"GVG_007",  # Flame Leviathan
	"AT_022",  # Fist of Jaraxxus
	"AT_130",  # Sea Reaver
)

_draftcache = {}


def _draft(card_class, exclude):
	# random_draft() is fairly slow, this caches the drafts
	if (card_class, exclude) not in _draftcache:
		_draftcache[(card_class, exclude)] = random_draft(card_class, exclude + BLACKLIST)
	return _draftcache[(card_class, exclude)], card_class.default_hero


_heroes = fireplace.cards.filter(collectible=True, type=CardType.HERO)


class BaseTestGame(CoinRules, BaseGame):
	def start(self):
		super().start()
		self.player1.max_mana = 10
		self.player2.max_mana = 10


def _random_class():
	return CardClass(random.randint(2, 10))


def _empty_mulligan(game):
	for player in game.players:
		if player.choice:
			player.choice.choose()


def init_game(class1=None, class2=None, exclude=(), game_class=BaseTestGame):
	log.info("Initializing a new game")
	if class1 is None:
		class1 = _random_class()
	if class2 is None:
		class2 = _random_class()
	player1 = Player("Player1", *_draft(class1, exclude))
	player2 = Player("Player2", *_draft(class2, exclude))
	game = game_class(players=(player1, player2))
	return game


def prepare_game(*args, **kwargs):
	game = init_game(*args, **kwargs)
	game.start()
	_empty_mulligan(game)

	return game


def prepare_empty_game(class1=None, class2=None, game_class=BaseTestGame):
	log.info("Initializing a new game with empty decks")
	if class1 is None:
		class1 = _random_class()
	if class2 is None:
		class2 = _random_class()
	player1 = Player("Player1", [], class1.default_hero)
	player1.cant_fatigue = True
	player2 = Player("Player2", [], class2.default_hero)
	player2.cant_fatigue = True
	game = game_class(players=(player1, player2))
	game.start()
	_empty_mulligan(game)

	return game

In [70]:
LORD_JARAXXUS = "EX1_323"
LORD_JARAXXUS_HERO = "EX1_323h"
LORD_JARAXXUS_WEAPON = "EX1_323w"
INFERNO = "EX1_tk33"
INFERNO_TOKEN = "EX1_tk34"


game = prepare_game(CardClass.WARRIOR, CardClass.WARRIOR)
game.player1.hero.power.use()
game.player1.give(LIGHTS_JUSTICE).play()
assert game.player1.weapon.id == LIGHTS_JUSTICE
game.end_turn()
game.end_turn()

print(game.player1.hand)
print(game.player2.hand)
print(game.player1.field)
#game.player1.give('EX1_414').play()
game.end_turn()
game.player2.give('EX1_414').play()
print(game.player1.field)
print(implemented_cards.index(game.player1.field[0]))
# assert game.player1.hero.health == 30
# assert game.player1.hero.armor == 2
# game.player1.give(LORD_JARAXXUS).play()
# assert game.player1.hero.id == LORD_JARAXXUS_HERO
# assert game.player1.weapon.id == LORD_JARAXXUS_WEAPON
# assert game.player1.hero.health == 15
# assert game.player1.hero.armor == 0
# assert game.player1.hero.power.id == INFERNO
# assert len(game.player1.field) == 0
# game.end_turn()
# game.end_turn()

# game.player1.hero.power.use()
# assert len(game.player1.field) == 1
# assert game.player1.field[0].id == INFERNO_TOKEN

[fireplace.1710115755]: Initializing a new game
[fireplace.entity]: Setting up game BaseTestGame(players=(Player(name='Player1', hero=None), Player(name='Player2', hero=None)))
[fireplace.entity]: Tossing the coin... Player2 wins!
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Summon(<Summon.CARD>=<HeroPower ('Armor Up!')>)> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Player1 summons [<HeroPower ('Armor Up!')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.actions]: Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>) triggering <TargetedAction: Summon(<Summon.CARD>=<Hero ('Garrosh Hellscream')>)> targeting [Player(name='Player1', hero=<Hero ('Garrosh Hellscream')>)]
[fireplace.actions]: Player1 summons [<Hero ('Garrosh Hellscream')>]
[fireplace.entity]: Empty stack, refreshing auras and processing deaths
[fireplace.entity]: Player(name

[<Minion ('Nerubian Unraveler')>, <Minion ('Twin Tyrant')>, <Minion ('Sir Finley Mrrgglton')>, <Minion ('Core Hound')>, <Minion ("Headmaster Kel'Thuzad")>]
[<Spell ('Charge')>, <Minion ('Theotar, the Mad Duke')>, <Minion ('Questing Adventurer')>, <Spell ('Brawl')>, <Spell ('The Coin')>, <Minion ('City Architect')>]
[]
[]


IndexError: list index out of range

In [71]:
game.player1.discard_hand()
print(game.player2.field)
game.end_turn()
mulch = game.player1.give('AT_044')
print(mulch.is_playable())
print(mulch.targets)
print(game.player1.hand)
mulch.play(target=mulch.targets[0])
print(game.player2.field)
print(game.player2.hand)

[fireplace.entity]: Player(name='Player2', hero=<Hero ('Garrosh Hellscream')>) discards their entire hand!
[fireplace.entity]: Discarding <Minion ("Headmaster Kel'Thuzad")>
[fireplace.card]: <Minion ("Headmaster Kel'Thuzad")> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <Minion ('Core Hound')>
[fireplace.card]: <Minion ('Core Hound')> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <Minion ('Sir Finley Mrrgglton')>
[fireplace.card]: <Minion ('Sir Finley Mrrgglton')> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <Minion ('Twin Tyrant')>
[fireplace.card]: <Minion ('Twin Tyrant')> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Discarding <Minion ('Nerubian Unraveler')>
[fireplace.card]: <Minion ('Nerubian Unraveler')> moves from <Zone.HAND: 3> to <Zone.GRAVEYARD: 4>
[fireplace.entity]: Player1 ends turn 4
[fireplace.entity]: Player2 begins turn 5
[fireplace.entity]: P

[<Minion ('Grommash Hellscream')>]
True
[<Minion ('Grommash Hellscream')>]
[<Minion ('Smothering Starfish')>, <Spell ('Mulch')>]
[]
[<Spell ('Charge')>, <Minion ('Theotar, the Mad Duke')>, <Minion ('Questing Adventurer')>, <Spell ('Brawl')>, <Spell ('The Coin')>, <Minion ('City Architect')>, <Minion ('Sir Finley Mrrgglton')>, <Minion ('Grim Necromancer')>]


In [77]:
print(game.player2.hand[7])
print(game.player2.hand[7].id)
print(game.player2.hand[7] in implemented_cards)
test_card = 'ICC_026'
print(test_card in implemented_cards)

Grim Necromancer
CORE_ICC_026
False
True


In [78]:
core_card = 'CORE_ICC_026'
print(core_card in implemented_cards)

False


In [79]:
card = 'YOD_012'
print(card in implemented_cards)

False


In [80]:
card = 'BAR_705'
print(card in implemented_cards)

False


In [1]:
import sys
print(sys.version)

3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
